# El poder de los decoradores

En esencia, los decoradores de Python permiten extender y modificar el comportamiento de una llamada (funciones, métodos y clases) sin necesidad de modificar la propia llamada.

Cualquier funcionalidad suficientemente genérica que se pueda añadir al comportamiento de una clase o función existente es un gran caso de uso para la decoración. 

Esto incluye lo siguiente:

* Registro de datos
* Aplicación de la autenticación de control de acceso 
* Instrumentación y limitación de las funciones
* Limitación de la velocidad
* Almacenamiento en caché, y más


¿Por qué deberías dominar el uso de los decoradores en Python? Después de todo, lo que acabo de mencionar suena bastante abstracto, y puede ser difícil ver cómo los decoradores pueden beneficiarte en tu trabajo diario como desarrollador de Python.

Los puntos más importantes de las "funciones de primera clase" para entender los decoradores son:

* Las funciones son objetos: Pueden asignarse a variables y pasarse a/ser devueltas por funciones.

* Las funciones pueden ser definidas dentro de otras funciones: Una función hijo puede capturar el estado local de su función padre (cierres léxicos).

## Conceptos básicos de los decoradores

Ahora bien, ¿qué son realmente los decoradores? 

**Los decoradores "decoran" o "envuelven" otra función y permiten ejecutar código antes y después de que se ejecute la función envuelta.

Los decoradores permiten definir bloques de construcción reutilizables que pueden cambiar o ampliar el comportamiento de otras funciones. Además, te permiten hacerlo sin modificar permanentemente la propia función envuelta. El comportamiento de la función sólo cambia cuando es decorada.

¿Cómo podría ser la implementación de un decorador simple? 

En términos básicos, un decorador es un objeto invocable que toma un objeto invocable como entrada y devuelve otro objeto invocable.

La siguiente función tiene esa propiedad y podría considerarse el decorador más sencillo que podrías escribir:

In [1]:
def decoradorNulo(funcion): 
    return funcion

Como puedes ver, decoradorNulo es un objeto invocable (callable, es una función), toma otro objeto invocable como entrada, y devuelve el mismo objeto invocable de entrada sin modificarlo.

In [2]:
def saludar():
    return 'Hola!'

saludar = decoradorNulo(saludar)

In [3]:
saludar()

En lugar de llamar explícitamente a decoradorNulo en saludar y luego reasignar la variable saludar, puedes usar la sintaxis @ de Python para decorar una función de forma más conveniente:

In [4]:
@decoradorNulo

def saludar():
    return 'Hola!'

In [5]:
saludar()

Poner una línea @decoradorNulo delante de la definición de la función es lo mismo que definir primero la función y luego pasar por el decorador. 

El uso de la sintaxis @ es sólo azúcar sintáctico y un atajo para este patrón comúnmente utilizado.

Tenga en cuenta que el uso de la sintaxis @ decora la función inmediatamente en el momento de la definición. 

Esto dificulta el acceso a la función original no decorada sin necesidad de realizar modificaciones sencillas. Por lo tanto, puedes elegir decorar algunas funciones manualmente para mantener la capacidad de llamar a la función no decorada también.

## Los decoradores pueden modificar el comportamiento

Ahora que estás un poco más familiarizado con la sintaxis del decorador, vamos a escribir otro decorador que realmente haga algo y modifique el comportamiento de la función decorada.

Aquí hay un decorador un poco más complejo que convierte el resultado de la función decorada en letras mayúsculas:

In [6]:
def mayusculas(funcion): 
    
    def envoltorio():
        
        return funcion().upper()
    
    return envoltorio

En lugar de devolver simplemente la función de entrada como hacía el decoradorNulo, este decorador de mayúsculas define una nueva función sobre la marcha (un cierre léxico) y la utiliza para envolver la función de entrada con el fin de modificar su comportamiento en el momento de la llamada.

El cierre envolvente tiene acceso a la función de entrada no decorada y es libre de ejecutar código adicional antes y después de llamar a la función de entrada. (Técnicamente, ni siquiera necesita llamar a la función de entrada).

Nótese que, hasta ahora, la función decorada nunca ha sido ejecutada. Llamar a la función de entrada en este momento no tendría ningún sentido: Querrás que el decorador sea capaz de modificar el comportamiento de su función de entrada cuando finalmente sea llamada.



In [7]:
@mayusculas
def saludar():
    return "Hola!"

In [8]:
saludar()

'HOLA!'

Al estar definido el decorador de esta forma, nunca se realiza la llamada a la función en su interior, ya que este devolverá un objeto en el cual está contenido la llamada a la función en vez de ejecutar la llamada él mismo.

In [9]:
mayusculas(saludar)

<function __main__.mayusculas.<locals>.envoltorio()>

El decorador de mayúsculas es una función en sí misma. Y la única manera de influir en el "comportamiento futuro" de una función de entrada que decora es reemplazar (o envolver) la función de entrada con un cierre.

Por eso mayusculas define y devuelve otra función (el cierre) que puede ser llamada en un momento posterior, ejecutar la función de entrada original y modificar su resultado.

Los decoradores modifican el comportamiento de una llamada a través de una capa envolvente (wrapper) para no tener que modificar permanentemente la original. La llamada original no se modifica permanentemente, su comportamiento sólo cambia cuando se decora.

## Aplicar multiples decoradores a una función

Tal vez no sea sorprendente que puedas aplicar más de un decorador a una función. Esto acumula sus efectos y es lo que hace que los decoradores sean tan útiles como bloques de construcción reutilizables.

He aquí un ejemplo. Los siguientes dos decoradores envuelven la cadena de salida de la función decorada en etiquetas HTML. 

Observando cómo se anidan las etiquetas, puedes ver qué orden utiliza Python para aplicar múltiples decoradores:

In [10]:
def strong(func): 
    def wrapper():
        return '<strong>' + func() + '</strong>' 
    return wrapper

def emphasis(func): 
    def wrapper():
        return '<em>' + func() + '</em>' 
    return wrapper

Ahora tomemos estos dos decoradores y apliquémoslos a nuestra función greet al mismo tiempo. Puedes usar la sintaxis @ normal para eso y simplemente "apilar" múltiples decoradores encima de una sola función:

In [11]:
@strong
@emphasis
def saludar():
    return "Hola!"

In [12]:
saludar()

'<strong><em>Hola!</em></strong>'

Esto muestra claramente en qué orden se aplicaron los decoradores: de abajo a arriba. Primero, la función de entrada fue envuelta por el decorador @emphasis, y luego la función resultante (decorada) fue envuelta de nuevo por el decorador @strong.

In [13]:
saludoDecorado = strong(emphasis(saludar))

In [14]:
saludoDecorado

<function __main__.strong.<locals>.wrapper()>

De nuevo, puedes ver que el decorador de énfasis se aplica primero y luego la función resultante envuelta es envuelta de nuevo por el decorador fuerte.

Esto también significa que los niveles profundos de apilamiento de decoradores tendrán incluso un efecto en el rendimiento porque siguen añadiendo llamadas a funciones anidadas. En la práctica, esto no suele ser un problema, pero es algo que hay que tener en cuenta si se trabaja con código de rendimiento intensivo que utiliza frecuentemente la decoración.

## Decorando funciones que aceptan argumentos

Todos los ejemplos hasta ahora sólo decoraban una simple función de saludo nula que no tomaba ningún argumento. Hasta ahora, los deco- radores que has visto aquí no tenían que lidiar con el envío de argumentos a la función de entrada.

Si intentas aplicar uno de estos decoradores a una función que toma argumentos, no funcionará correctamente. ¿Cómo decorar una función que toma argumentos arbitrarios?

Aquí es donde la función \*args y \**kwargs de Python3 para tratar con números variables de argumentos es muy útil. 

El siguiente decorador proxy se aprovecha de ello:

In [15]:
def proxy(func):
    
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) 
    
    return wrapper

Hay dos cosas notables en este decorador:

* Utiliza los operadores * y** en la definición del envoltorio para recoger todos los argumentos posicionales y de palabra clave y los almacena en variables (args y kwargs).

* El envoltorio devuelve los argumentos recogidos a la función de entrada original utilizando los operadores * y ** de "desempacado de argumentos".

Es un poco desafortunado que el significado de los operadores estrella y doble estrella esté sobrecargado y cambie dependiendo del contexto en el que se usen, pero espero que se entienda la idea.

Ampliemos la técnica expuesta por el decorador proxy a un ejemplo práctico más útil. Aquí tenemos un decorador de rastreo que registra los argumentos de la función y los resultados durante el tiempo de ejecución:

In [16]:
def trace(func):
    
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() ' 
            f'with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs) 
        
        print(f'TRACE: {func.__name__}() '
              f'returned {original_result!r}')
        
        return original_result 
    
    return wrapper

Al decorar una función con trace y luego llamarla, se imprimirán los argumentos pasados a la función decorada y su valor de retorno. Esto sigue siendo un ejemplo "de juguete", pero en un apuro es una gran ayuda para la depuración:

In [17]:
@trace
def decir(nombre, linea):
    return f'{nombre}: {linea}'

In [18]:
decir("Carlos", "Hola mundo")

TRACE: calling decir() with ('Carlos', 'Hola mundo'), {}
TRACE: decir() returned 'Carlos: Hola mundo'


'Carlos: Hola mundo'

## Como escribir decoradores depurables 

Cuando se utiliza un decorador, en realidad lo que se hace es sustituir una función por otra. Una de las desventajas de este proceso es que "oculta" algunos de los metadatos adjuntos a la función original (no decorada).

Por ejemplo, el nombre de la función original, su docstring y la lista de parámetros quedan ocultos por el cierre de la envoltura:

In [19]:
def saludar():
    """Devuelve un saludo"""
    return "Hola!"

In [20]:
saludoDecorado = mayusculas(saludar)

Si intentas acceder a los metadatos, ves que apareceran los metadatos del envoltorio (wrapper) en vez de los de la función saludar, ya que es lo que se devuelve.

In [21]:
saludar.__name__

'saludar'

In [22]:
saludar.__doc__

'Devuelve un saludo'

In [23]:
saludoDecorado.__name__

'envoltorio'

In [24]:
saludoDecorado.__doc__

Esto hace que la depuración y el trabajo con el intérprete de Python sean incómodos y desafiantes. 

Afortunadamente hay una solución rápida para esto: **el decorador functools.wraps incluido en la biblioteca estándar de Python.**

Puedes utilizar functools.wraps en tus propios decoradores para copiar los metadatos perdidos de la función no decorada al cierre del decorador. 

He aquí un ejemplo:

In [25]:
import functools

def uppercase(func): 
    
    @functools.wraps(func) 
    def wrapper():
        return func().upper() 
    
    return wrapper

La aplicación de functools.wraps al cierre envolvente devuelto por el decorador transporta el docstring y otros metadatos de la función de entrada:

In [26]:
@uppercase
def saludar():
    """Devuelve un saludo"""
    return "Hola!"

In [27]:
saludar.__name__

'saludar'

In [28]:
saludar.__doc__

'Devuelve un saludo'

### Consejo

Como mejor práctica, te recomiendo que utilices functools.wraps en todos los decoradores que escribas tú mismo. No lleva mucho tiempo y te ahorrará (y a otros) dolores de cabeza de depuración en el futuro.

## Claves

* Los decoradores definen bloques de construcción reutilizables que puedes aplicar a una llamada para modificar su comportamiento sin modificar permanentemente la propia llamada.

* La sintaxis @ es sólo una abreviatura para llamar al decorador en una función de entrada. Los decoradores múltiples en una sola función se aplican de abajo hacia arriba (apilamiento de decoradores).

* Como mejor práctica de depuración, utiliza el ayudante de functools.wraps en sus propios decoradores para transferir los metadatos de la función no decorada a la decorada.

* Al igual que cualquier otra herramienta en la caja de herramientas de desarrollo de software, los decoradores no son una cura para todo y no deben ser utilizados en exceso. Es importante equilibrar la necesidad de "hacer cosas" con el objetivo de "no enredarse en un lío horrible e imposible de mantener de una base de código".